# Sprint 4: Context Filter Analysis

**Ziel:** Analyse der Context Filter Performance und Threshold-Kalibrierung

## Überblick

- **Implemented Filters:**
  - `detect_negation()` - Negierte Krisen-Begriffe
  - `detect_reported_speech()` - Zitate/Berichte (0.3-0.6x)
  - `detect_hypothetical()` - Hypothetische Konstruktionen (0.5x)

- **Aktuelles Problem:**
  - C4_Positive_Thanks: 0.235 (sollte <0.10 sein)
  - Root Cause: "Hilfe" als Panik-Keyword ohne Kontext

---

In [ ]:
# Setup
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add backend to path
backend_path = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(backend_path))

# Import modules
from core.evoki_metrics_v3 import emotions as emo
from core.evoki_metrics_v3 import crisis_scoring as crisis

# Style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("✅ Modules loaded successfully")

---

## 1. Load Trigger Analysis Results

Laden der CSV-Ergebnisse aus `generate_trigger_analysis.py`

In [ ]:
# Load results
df = pd.read_csv('../trigger_analysis_full_metrics.csv')

# Separate crisis and control prompts
crisis_df = df[~df['TestCase'].str.startswith('C')].copy()
control_df = df[df['TestCase'].str.startswith('C')].copy()

print(f"📊 Total Test Cases: {len(df)}")
print(f"   - Crisis prompts: {len(crisis_df)}")
print(f"   - Control prompts: {len(control_df)}")
print(f"\n🔥 Max Control Score: {control_df['crisis_score'].max():.3f}")
print(f"⚠️  Target: <0.10\n")

# Display control scores
print("Control Prompt Scores:")
print(control_df[['TestCase', 'crisis_score', 'fear']].to_string(index=False))

---

## 2. C4 False Positive Deep Dive

Detaillierte Analyse warum C4_Positive_Thanks zu hoch scored

In [1]:
# Test text
c4_text = "Vielen Dank für deine Hilfe. Das hat mir sehr geholfen."

print("="*70)
print("C4 FALSE POSITIVE ANALYSIS")
print("="*70)
print(f"\nText: {c4_text}\n")

# Compute all emotions
emotions = {
    'joy': emo.compute_m77_joy(c4_text),
    'sadness': emo.compute_m78_sadness(c4_text),
    'anger': emo.compute_m79_anger(c4_text),
    'fear': emo.compute_m80_fear(c4_text),
    'trust': emo.compute_m81_trust(c4_text),
    'disgust': emo.compute_m82_disgust(c4_text),
    'anticipation': emo.compute_m83_anticipation(c4_text),
    'surprise': emo.compute_m84_surprise(c4_text),
}

print("Emotion Scores:")
for emotion, score in emotions.items():
    status = "⚠️" if score > 0.15 else "✓"
    print(f"  {status} {emotion:15s}: {score:.3f}")

# Test context filters
speech_factor = crisis.detect_reported_speech(c4_text)
hypo_factor = crisis.detect_hypothetical(c4_text)

print(f"\nContext Filters:")
print(f"  Reported Speech Factor: {speech_factor:.2f}")
print(f"  Hypothetical Factor:    {hypo_factor:.2f}")

# Compute crisis score
confusion = emo.compute_m87_confusion(c4_text)
crisis_score, category = crisis.compute_crisis_auto(
    'C4_Positive_Thanks',
    emotions['sadness'],
    emotions['fear'],
    emotions['anger'],
    emotions['joy'],
    emotions['surprise'],
    emotions['trust'],
    confusion,
    c4_text
)

# Calculate base score (without filters)
crisis_base = max(emotions['sadness'], emotions['fear'])

print(f"\nCrisis Scoring:")
print(f"  Category Detected:  {category}")
print(f"  Base Score:        {crisis_base:.3f}")
print(f"  After Filters:     {crisis_score:.3f}")
print(f"  Reduction:         {(1 - crisis_score/crisis_base)*100 if crisis_base > 0 else 0:.1f}%")

print(f"\n❌ PROBLEM: Fear = {emotions['fear']:.3f} (triggered by 'Hilfe')")
print(f"   Filters don't help because 'Hilfe' is in direct speech.")

C4 FALSE POSITIVE ANALYSIS

Text: Vielen Dank für deine Hilfe. Das hat mir sehr geholfen.



NameError: name 'emo' is not defined

In [ ]:
# TODO: clarify the requested operation

---

## 3. Detection Rate Analysis

Performance bei verschiedenen Thresholds

In [ ]:
# Calculate detection rates at different thresholds
thresholds = [0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.50]

results = []
for thresh in thresholds:
    # Crisis detection
    crisis_detected = (crisis_df['crisis_score'] > thresh).sum()
    crisis_rate = crisis_detected / len(crisis_df) * 100
    
    # Control false positives
    control_fp = (control_df['crisis_score'] > thresh).sum()
    control_fp_rate = control_fp / len(control_df) * 100
    
    # Precision (TP / TP+FP)
    tp = crisis_detected
    fp = control_fp
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    results.append({
        'Threshold': thresh,
        'Detection_Rate': crisis_rate,
        'False_Positive_Rate': control_fp_rate,
        'Precision': precision
    })

results_df = pd.DataFrame(results)
print("\n📊 DETECTION RATES BY THRESHOLD\n")
print(results_df.to_string(index=False))

# Find optimal threshold
# Target: FP < 10% (0.4/4 = 10%) AND highest detection rate
optimal = results_df[results_df['False_Positive_Rate'] < 25.01]  # 1/4 = 25%
if len(optimal) > 0:
    best_thresh = optimal.loc[optimal['Detection_Rate'].idxmax(), 'Threshold']
    print(f"\n✅ Optimal Threshold: {best_thresh:.2f}")
    print(f"   (Maximizes detection while FP < 25%)")
else:
    print("\n⚠️  No threshold achieves FP < 25%")

---

## 4. Visualizations

In [ ]:
# Plot 1: Detection Rate vs Threshold
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left: Detection & FP rates
ax1.plot(results_df['Threshold'], results_df['Detection_Rate'], 
         'o-', linewidth=2, markersize=8, label='Detection Rate (Crisis)', color='#2ecc71')
ax1.plot(results_df['Threshold'], results_df['False_Positive_Rate'], 
         's-', linewidth=2, markersize=8, label='False Positive Rate (Control)', color='#e74c3c')
ax1.axhline(y=25, color='#e74c3c', linestyle='--', alpha=0.5, label='Target FP < 25%')
ax1.set_xlabel('Crisis Score Threshold', fontsize=12, fontweight='bold')
ax1.set_ylabel('Rate (%)', fontsize=12, fontweight='bold')
ax1.set_title('Detection vs False Positive Rate', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Right: Precision
ax2.plot(results_df['Threshold'], results_df['Precision'] * 100, 
         'D-', linewidth=2, markersize=8, color='#3498db')
ax2.axhline(y=85, color='#2ecc71', linestyle='--', alpha=0.5, label='Target Precision > 85%')
ax2.set_xlabel('Crisis Score Threshold', fontsize=12, fontweight='bold')
ax2.set_ylabel('Precision (%)', fontsize=12, fontweight='bold')
ax2.set_title('Precision at Different Thresholds', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../sprint4_threshold_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Saved: sprint4_threshold_analysis.png")

In [ ]:
# Plot 2: Score Distribution (Crisis vs Control)
fig, ax = plt.subplots(figsize=(14, 7))

# Crisis scores
ax.hist(crisis_df['crisis_score'], bins=20, alpha=0.6, 
        label=f'Crisis Prompts (n={len(crisis_df)})', color='#e74c3c', edgecolor='black')

# Control scores  
ax.hist(control_df['crisis_score'], bins=20, alpha=0.8,
        label=f'Control Prompts (n={len(control_df)})', color='#2ecc71', edgecolor='black')

# Threshold markers
for thresh in [0.15, 0.25, 0.30]:
    ax.axvline(x=thresh, color='blue', linestyle='--', alpha=0.5, linewidth=1.5)
    ax.text(thresh, ax.get_ylim()[1]*0.95, f'{thresh:.2f}', 
            ha='center', fontsize=10, fontweight='bold')

ax.set_xlabel('Crisis Score', fontsize=12, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax.set_title('Crisis Score Distribution: Crisis vs Control Prompts', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../sprint4_score_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Saved: sprint4_score_distribution.png")

---

## 5. Context Filter Effectiveness Test

Test der Filter mit verschiedenen Beispielen

In [ ]:
# Test cases for context filters
test_cases = {
    'Direct Crisis': "Ich will sterben. Es ist zu viel.",
    'Reported Speech': 'Er sagte: "Ich will sterben"',
    'Hypothetical': "Was wäre wenn ich sterbe? Wäre das besser?",
    'Negated': "Ich will NICHT sterben. Das Leben ist schön.",
    'C4 (Problem)': "Vielen Dank für deine Hilfe. Das hat mir sehr geholfen.",
}

print("\n" + "="*70)
print("CONTEXT FILTER EFFECTIVENESS TEST")
print("="*70 + "\n")

for name, text in test_cases.items():
    # Compute filters
    speech = crisis.detect_reported_speech(text)
    hypo = crisis.detect_hypothetical(text)
    
    # Compute fear (main contributor)
    fear = emo.compute_m80_fear(text)
    sadness = emo.compute_m78_sadness(text)
    
    # Base vs filtered
    base = max(fear, sadness)
    filtered = base * speech * hypo
    
    print(f"📝 {name}")
    print(f"   Text: {text}")
    print(f"   Fear: {fear:.3f} | Sadness: {sadness:.3f}")
    print(f"   Speech Filter: {speech:.2f}x | Hypo Filter: {hypo:.2f}x")
    print(f"   Base Score: {base:.3f} → Filtered: {filtered:.3f} ({(1-filtered/base)*100 if base > 0 else 0:.0f}% reduction)")
    print()

---

## 6. Proposed Solutions for C4

### Option 1: Threshold Adjustment
- Raise threshold from 0.15 → 0.25
- Pro: Simple, no code changes
- Con: May reduce detection rate

### Option 2: Lexikon Refinement
- Add context-aware weights for "Hilfe"
- Only trigger in negative context
- Pro: Addresses root cause
- Con: Requires lexikon restructuring

### Option 3: Lexikon-Level Negation
- Integrate negation detection into `compute_lexicon_score()`
- Pro: Catches all lexikon-based false positives
- Con: Higher complexity

---

In [ ]:
# Simulate Option 1: Threshold Adjustment
print("\n" + "="*70)
print("OPTION 1 SIMULATION: Threshold = 0.25")
print("="*70 + "\n")

new_thresh = 0.25

# Recalculate with new threshold
crisis_detected = (crisis_df['crisis_score'] > new_thresh).sum()
control_fp = (control_df['crisis_score'] > new_thresh).sum()

print(f"Detection Rate: {crisis_detected}/{len(crisis_df)} ({crisis_detected/len(crisis_df)*100:.1f}%)")
print(f"False Positives: {control_fp}/{len(control_df)} ({control_fp/len(control_df)*100:.1f}%)")
print(f"Precision: {crisis_detected/(crisis_detected+control_fp)*100:.1f}%")

if control_fp == 0:
    print("\n✅ SUCCESS: No false positives!")
else:
    print(f"\n⚠️  Still {control_fp} false positive(s)")
    print(control_df[control_df['crisis_score'] > new_thresh][['TestCase', 'crisis_score']])

---

## 7. Recommendations

Based on the analysis:

1. **Short-term fix:** Raise threshold to 0.25
   - Eliminates C4 false positive
   - Maintains acceptable detection rate

2. **Medium-term:** Implement lexikon-level context awareness
   - Add positive/negative context detection
   - Weight "Hilfe" based on surrounding sentiment

3. **Long-term:** Full semantic context analysis
   - Use embeddings for context understanding
   - ML-based sentiment disambiguation

---

In [ ]:
# Export results
summary = {
    'sprint': 'Sprint 4',
    'filters_implemented': ['negation', 'reported_speech', 'hypothetical'],
    'current_threshold': 0.15,
    'proposed_threshold': 0.25,
    'c4_score_current': control_df[control_df['TestCase'] == 'C4_Positive_Thanks']['crisis_score'].values[0],
    'detection_rate_015': results_df[results_df['Threshold'] == 0.15]['Detection_Rate'].values[0],
    'detection_rate_025': results_df[results_df['Threshold'] == 0.25]['Detection_Rate'].values[0],
    'fp_rate_015': results_df[results_df['Threshold'] == 0.15]['False_Positive_Rate'].values[0],
    'fp_rate_025': results_df[results_df['Threshold'] == 0.25]['False_Positive_Rate'].values[0],
}

print("\n📊 SPRINT 4 SUMMARY")
print("="*70)
for key, value in summary.items():
    print(f"{key:25s}: {value}")

print("\n✅ Analysis complete!")